# フリーハンド入力した手書き数字の認識

---

一般に機械学習アプリケーションは、ユーザとのインターラクションのためのフロントエンドと認識システムとを一体化する構成が多いと思われます。しかし、本テンプレートででは簡単化のために Jupyter Notebook をフロントエンド環境として使用し、認識は先に構築した認識システムを使用します。

本Notebookでは、フロントエンドで入力した手書き数字をイメージファイルとして認識システムに転送、認識システムで認識し、結果をフロントエンドに戻します。

![構成](images/230.frontend.png)

## 認識システム設定

使用する認識システムの `UnitGroup` 名を指定します。なお、内部的には UnitGroup 名のついたファイル名をアクセスするだけで、VCP へはアクセスしません。

In [ ]:
rs_ugroup_name = 'TfCpu'

## 準備

### 認識、学習システムのパラメータ読み出し

認識システムのパラメータを読み出します。学習システムを構築した場合(trainingたsystem = True)学習システムのパラメータも読み出します。次の場合エラーとなります。
* training_system = True かつ学習システム構築Notebookの作業フォルダ({ohpc_WORK_DIR})が存在しない
* training_system = True かつ学習システムの UnitGroup 名({ts_ugroup_name})が存在しない

学習システムを構築していない場合(training_system = False)、あらかじめ用意してあるモデルと重みを作業領域にコピーします。

In [ ]:
# 学習済みモデル・重みファイル
fe_save_dir = './data'            # フロントエンド重み保存フォルダ
model_file = 'saved_model.tgz'    # ファイル名

### 構築時パラメータの読み込み
%run scripts/ts/group.py
rs_gvars = load_group_vars(rs_ugroup_name)
print(rs_gvars)
if rs_gvars['training_system']:
    ts_gvars = load_group_vars(rs_gvars['ts_ugroup_name'], "../1.learning_system/" + rs_gvars['ohpc_WORK_DIR'])
    print(ts_gvars)
else:
    !cp -p ./original/model/{model_file} {fe_save_dir}/{model_file}

### 内部パラメタの設定

In [ ]:
### 学習システム、認識システム共通 ###############################
# 作業ディレクトリ（学習システム、認識システムとも同じパス）
tf_work_dir = rs_gvars['tf_work_dir']


### 認識システム ##############################################
# コマンドラインオプション
rs_user = rs_gvars['rs_user']
rs_user_ssh_opts = f'-i {rs_gvars["rs_user_prvkey"]}'
rs_target = f'{rs_user}@{rs_gvars["rs_ipaddress"]}'     # 認識システム構築時の ipaddress を使用。停止・再起動未対応


### 学習システム ##############################################
# 学習システムを構築していない場合は、認識システムを使用
if rs_gvars['training_system']:
    ts_user = ts_gvars['ohpc_user']
    ts_user_ssh_opts = f'-i {ts_gvars["ohpc_user_prvkey"]}'
    ts_target = f'{ts_user}@{ts_gvars["master_ipaddress"]}'
else:
    ts_user = rs_user
    ts_user_ssh_opts = rs_user_ssh_opts
    ts_target = rs_target


### フロントエンド（本Jupyter Notebook）############
# 認識スクリプトを格納しているディレクトリ
scripts = './scripts/recognition'

# フリーハンド入力した数字を格納するイメージファイル名（Jupyter上）
img_file = f'{fe_save_dir}/base64_img.b64'

### 学習結果を認識システムへ転送、展開

学習システムで学習したモデルと学習結果を認識システムに転送し展開します。

In [ ]:
# 学習結果を認識システムにアップロードし展開
!ssh {rs_user_ssh_opts} {rs_target} rm -rf {tf_work_dir}/{model_file} {tf_work_dir}/saved_model
!scp {rs_user_ssh_opts} -qp {fe_save_dir}/{model_file} {rs_target}:{tf_work_dir}
!ssh {rs_user_ssh_opts} {rs_target} '(cd {tf_work_dir}; tar zxpf {model_file})'

### フリーハンド認識の準備

In [ ]:
import numpy as np
from IPython.core.display import HTML
import base64
from io import BytesIO
from PIL import Image
from PIL import ImageOps
import matplotlib.pyplot as plt

def recognise(image):
    # イメージをファイルに保存
    with open(img_file, "w") as f1:
        f1.write(image)
    
    # 確認のため MNIST のイメージサイズ(28x28)に変換して、拡大表示
    img = Image.open(BytesIO(base64.b64decode(base64_img.split(",")[-1]))).resize((28,28))
    plt.imshow(np.asarray(img))
    plt.show()

    # VCノードへイメージを転送し認識
    !scp {rs_user_ssh_opts} -qp {img_file} {rs_target}:{tf_work_dir}
    !ssh {rs_user_ssh_opts} {rs_target} docker exec -t -u {rs_user} -w /home/{rs_user}/{tf_work_dir} tensorflow-{rs_user} python3 mnist_predict.py

# raise Exception("\nStopped")

### バッチジョブ操作関数の定義

In [ ]:
# バッチジョブ投入関数定義(slurm 用)
def qsub(job, work_dir="."):
    bs_state = !ssh {ts_user_ssh_opts} {ts_target} bash -l -c "'cd {work_dir} && sbatch {job}'"
    bs_state = bs_state[len(bs_state)-1]
    print(bs_state)

    if bs_state.split()[0] == 'Submitted':
        return bs_state.split()[3]
    else:
        raise RuntimeError('ERROR: job submission error')

        
def qstat(bs_jobid):
    print('job ' + bs_jobid + ':', end = ' ')
    while True:
        bs_state = !ssh {ts_user_ssh_opts} {ts_target} bash -l -c 'squeue --noheader --jobs={bs_jobid}'
        bs_state = bs_state[len(bs_state)-1]
        if bs_state == '' or bs_state.split()[0] == 'JOBID':
            break
        elif 'error' in bs_state:
            raise RuntimeError(bs_state)
        else:
            print("", end = '.')
            !sleep 5
    print(' finished')

## フリーハンド数字入力と認識
数字（一桁の'0-9'）をフリーハンドで入力します。マウスをクリックしながら描画してください。描画が終わったら、`Save`ボタンをクリックし、２つ目のセルを実行してください。入力したイメージをVCノードのTensorFlow環境に転送し認識します。

１つ目のセル（入力環境）は２つ目のセルを実行した後も実行が継続されています。`Clear`で描画エリアをクリア後、再度入力し`Save`をクリックしてください。

２つ目のセルは凍結を解除し再実行することで再入力した数字を認識します。

In [ ]:
# フリーハンド数字入力、base64形式イメージ化
%run scripts/frontend/fhinput.py
HTML(fhinput())

In [ ]:
recognise(base64_img)

## 誤答が多い？　GPU 資源は利用料が高い？　いろいろ試してみましょう。
学習時のテストでは97%以上の認識精度が出ているのに、誤認識が多いと思います。改善のためいろいろ試してみます。そして、コスト低減を考慮し認識システムでの学習を試します。

### 一般的な機械学習ベースの認識システムの誤認識対応

アプリ運用中に誤答が多くなると、誤答データ（イメージ）と正解ラベルを学習システムに戻して追加学習し、誤答に対応するのが一般的です。
しかし、その前にこの認識システムには基本的な問題があります。

### ニューラルネットワークモデルは適切か？

このNotebookのMNISTのニューラルネットワークのモデルは、TensorFlowホームページの「初心者のための TensorFlow 2.0 入門」を基にしており、フラットな４層ニューラルネットワークです（scripts/tensorflow/mnist_train.py）。

```
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])
```

一般に、フラットなモデル（ある層の全てのユニットが前の層の全てのユニットに接続する）で高い認識精度を求めるするのは非常に難しいと考えられます。学習時のテストで97%以上の高い精度を示したのは、テストデータの中に学習データと同じ筆者のデータが含まれているためと思われます。もう一つ重要な理由がありますが、それは後ほど。

画像認識では、畳み込みニューラルネットワーク（CNN）がよく使われます。フラットなモデルとの違いは、

* フラット: 各層でイメージまたは前層の出力全体を認識します
* CNN: 畳み込み層はイメージまたは前層の出力の小領域の部分特徴を認識します。次の畳み込み層では前層の部分特徴の組み合わせをさらに部分特徴として認識します（結果として、前層より入力層の大きな部分を見ることになります）。これを積み重ねることにより、最終的に全体を認識します

Tensorflow の CNN とは異なりますが、畳み込みによる文字認識の仕組みがわかりやすいので、ネオコグニトロンの認識の仕組みを示します。

![構成](images/231.neocognitron.png)

部分的なパターンの認識結果を次の層で組み合わせます。このとき前の層のパターンに部分的な変形や位置がずれがあっても、層を重ねるにつれ徐々に影響が薄れ、最終的には影響を受けにくくなります。フラットなモデルの場合、前の層のパターン全体を見るので、変形や位置ずれの影響が最終層まで残りやすくなっています。なお、サル（おそらく人間も）の一次視覚野には、角度別の微小な線分に反応する細胞が存在することが生理学的に確認されています。

では、CNNで学習して認識精度を見てみましょう。モデルは以下の通りです（scripts/tensorflow/mnist_training2.py）。

```
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (5,5), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
```

CNN で学習するためのジョブを投入します。学習システムを構築していない場合は実行せず、あらかじめ用意している CNN での学習結果を使用します。

In [ ]:
if rs_gvars['training_system']:
    # 以前の学習ジョブ結果の削除
    !ssh {ts_user_ssh_opts} {ts_target} bash -l -c \
        "'cd {tf_work_dir} && rm -rf  tensorflow-mnist*.out saved_model*'"

    # 学習ジョブの投入と実行終了待ち
    jobid = qsub('tensorflow_mnist_docker_ho2.job', tf_work_dir)
    qstat(jobid)
    
    # 学習結果の表示。学習時の認識精度はフラットなニューラルネットより向上しているのがわかります（97.5%程度→99%程度)
    !ssh {ts_user_ssh_opts} {ts_target} bash -l -c \
        "'cd {tf_work_dir} && tail tensorflow-mnist*.out'"
    
else:
    !cp -p ./original/model/saved_model2.tgz {fe_save_dir}/{model_file}

学習結果（学習モデルと重み）を、認識システムに展開します。

In [ ]:
# 学習システムでの学習結果をフロントエンドにダウンロード
!scp {ts_user_ssh_opts} -p {ts_target}:{tf_work_dir}/{model_file} {fe_save_dir}

# 学習結果を認識システムにアップロードし展開
!ssh {rs_user_ssh_opts} {rs_target} rm -rf {tf_work_dir}/{model_file} {tf_work_dir}/saved_model
!scp {rs_user_ssh_opts} -qp {fe_save_dir}/{model_file} {rs_target}:{tf_work_dir}
!ssh {rs_user_ssh_opts} {rs_target} '(cd {tf_work_dir}; tar zxpf {model_file})'

上のフリーハンド入力のセルに戻って、試してみてください。
多少改善されているはずです。確認したら戻って次の節を試してください。

### ニューラルネットワークに与えるデータは適切か？

人間でも真正面の文字を認識するときと、視線の中心から少し外れた文字を認識するときとでは、後者の方が誤認識が多くなると思います。これは人工的なニューラルネットでも同じです。本当は、上記のようなニューラルネットのモデルの違いより、画像が認識領域の中心にあるか否かが大きく影響します。そこで、入力システムに描いた画像を中心に移動する機能を追加しました。試してみてください。

`Save`ボタンで従来通り保存して認識してみてください。次に画像を変えずに`Centralize and save`で画像を中心に配置して保存して認識してみてください。上の入力システムと同様に、２つ目のセル（認識）を実行した後も機能しています。

In [ ]:
# フリーハンド数字入力、base64形式イメージ化
%run scripts/frontend/fhinput.py
HTML(fhinput2())

In [ ]:
recognise(base64_img)

かなり、認識精度が向上したと思います。本Notebookでの認識精度の向上はここまでとします。

なお、今回使用したCNNは４層です（CNNは最初の2層のみ）。ニューラルネットは層が多いほど複雑な動作ができます。例えば、AND や OR は中間層のない１層で実現できますが、さらに複雑な動作である XOR は少なくとも１つの中間層が必要です（２層モデル）。このように、多層になるほど複雑な動作ができると考えられるので、フラットなニューラルネットでも層数を増やせば精度が上がるかもしれません。ちなみに、認識精度の高いニューラルネットの多くはCNNを使っていますが１００層を超えます。

興味ある方は ./scripts/recognition の mnist_train.py や mnist_train_cnn.py を編集して試してみてください。

### 利用している資源のspecは適切か？

今回は[OpenHPC-v2](https://github.com/nii-gakunin-cloud/ocs-templates/tree/master/OpenHPC-v2)と同様に GPU 資源を使いましたが、MNIST 程度の小規模データ、小規模ニューラルネットワークモデルでは、CPU だけの環境でも学習するのにさほど時間はかかりません。認識システムとして構築した CPU のみの環境で学習させてみます。本節の学習結果による認識は、前節の入力システムで確認できます。

#### フラットなニューラルネットワークを CPU のみの環境で学習
学習は１分以内に終了するはずです。なお、認識システムの実行環境は、ジョブスケジューラではなくインタラクティブ環境のためセルの実行の終了が学習の終了です。

In [ ]:
!ssh {rs_user_ssh_opts} {rs_target} rm -rf {tf_work_dir}/saved_model
!ssh {rs_user_ssh_opts} {rs_target} docker exec -t -u {rs_user} -w /home/{rs_user}/{tf_work_dir} tensorflow-{rs_user} python3 download_mnist.py
!ssh {rs_user_ssh_opts} {rs_target} docker exec -t -u {rs_user} -w /home/{rs_user}/{tf_work_dir} tensorflow-{rs_user} python3 mnist_train.py

#### CNN を CPU のみの環境で学習
学習は８分程度で終了するはずです。時間に余裕がある方は試してみてください。

In [ ]:
!ssh {rs_user_ssh_opts} {rs_target} rm -rf {tf_work_dir}/saved_model
!ssh {rs_user_ssh_opts} {rs_target} docker exec -t -u {rs_user} -w /home/{rs_user}/{tf_work_dir} tensorflow-{rs_user} python3 download_mnist.py
!ssh {rs_user_ssh_opts} {rs_target} docker exec -t -u {rs_user} -w /home/{rs_user}/{tf_work_dir} tensorflow-{rs_user} python3 mnist_train_cnn.py